In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import scanpy as sc
from glob import glob
import pandas as pd
import numpy as np
import seaborn as sns
import anndata
import scipy
import sklearn
import SpatialDE
import math
import re
import os
import random
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import rcParams
from matplotlib.colors import rgb2hex
import matplotlib.cm as cm
from matplotlib.legend import Legend
%matplotlib inline
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)
pd.options.display.max_seq_items = 2000

sc.set_figure_params(scanpy=True, dpi=300, dpi_save=1200, frameon=True, vector_friendly=True, fontsize=4, 
                         color_map='Dark2', format='pdf', transparent=True, ipython_format='png2x')

rcParams.update({'font.size': 4})
rcParams.update({'font.family': 'Helvetica'})
rcParams['pdf.fonttype'] = 42
rcParams['ps.fonttype'] = 42

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from IPython.core.display import display, HTML
display(HTML("""
<style>
#notebook-container {
    width: 100%
}
 
.code_cell {
   flex-direction: row !important;
}
 
.code_cell .input {
    width: 60%
}
 
.code_cell .output_wrapper {
    width: 40%
}
</style>
"""))

In [ ]:
## load combined data

samples = ['A1','B1','C1','D1']

for i,s in enumerate(samples):

    tmp = sc.read_visium(path='/Users/gouink/Documents/Bladder/Visium/'+s+'_pre/',library_id=s)
    tmp.var_names_make_unique()
    tmp.obs['sampleID'] = s
    
    if (s=='A1'):
        tmp.obs['Patient'] = 'Bladder1204'
    elif (s=='B1'):
        tmp.obs['Patient'] = 'Bladder8'
    elif (s=='C1'):
        tmp.obs['Patient'] = 'Bladder72'
    elif (s=='D1'):
        tmp.obs['Patient'] = 'Bladder371'
    
    ## remove part of C1 that has no tissue
    if (s == 'C1'):
        tmp = tmp[~((tmp.obs.array_row>=30)&(tmp.obs.array_row<=38)&(tmp.obs.array_col<=80)&(tmp.obs.array_col>=65))]
    
    if i == 0:
        adata = tmp
    else:
        adata = adata.concatenate(tmp,batch_key=None,uns_merge='unique')

adata.var["mt"] = adata.var_names.str.startswith("MT-")
sc.pp.calculate_qc_metrics(adata, qc_vars=["mt"], inplace=True)

sc.pp.filter_cells(adata, min_counts=1250)
adata = adata[adata.obs["pct_counts_mt"] < 10]
print(f"#cells after MT filter: {adata.n_obs}")
sc.pp.filter_genes(adata, min_cells=4)
sc.pp.normalize_total(adata,target_sum=10000,inplace=True)
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata, flavor="seurat", n_top_genes=5000)

In [ ]:
## spatialDE

for i,s in enumerate(samples):
    tmp = adata[(adata.obs.sampleID==s),:].copy()
    counts = pd.DataFrame(tmp.X.todense(), columns=tmp.var_names, index=tmp.obs_names)
    coord = pd.DataFrame(tmp.obsm['spatial'], columns=['x_coord', 'y_coord'], index=tmp.obs_names)
    results = SpatialDE.run(coord, counts)
    results.index = results["g"]
    tmp.var = pd.concat([tmp.var, results.loc[results.index.intersection(tmp.var.index.values), :]], axis=1)
    results.sort_values("qval").to_csv('/Users/gouink/Documents/Bladder/Visium/'+s+'_pre_spatialDEoutput.csv')

In [ ]:
# derive gene correlation modules from visium data

for i,s in enumerate(samples):
    results = pd.read_csv('/Users/gouink/Documents/Bladder/Visium/'+s+'_pre_spatialDEoutput.csv')
    results = results[(results.qval<=0.05)]
    if i == 0:
        GOI = results.index.tolist()
    else:
        GOI = GOI + results.index.tolist()

GOI = GOI + adata.var.index[(adata.var.highly_variable)].tolist()

GOI = np.unique(GOI)
GOI = [x for x in GOI if not re.search('^AC',x)]
GOI = [x for x in GOI if not re.search('\.[0-9]$',x)]
GOI = [x for x in GOI if not re.search('-AS[0-9]',x)]
GOI = [x for x in GOI if not re.search('RP',x)]
GOI = [x for x in GOI if not re.search('LINC',x)]
GOI = [x for x in GOI if x in adata.var.index]
len(GOI)

corrMat = np.corrcoef(adata[:,GOI].X.toarray(),rowvar=False)
corrMat = pd.DataFrame(corrMat,index=GOI,columns=GOI)

numcorrs = 5
corrcutoff = 0.4

corrMat = corrMat.loc[np.sum(abs(corrMat.to_numpy())>corrcutoff,axis=1)>numcorrs,np.sum(abs(corrMat.to_numpy())>corrcutoff,axis=1)>numcorrs]
corrMat.shape

d = scipy.spatial.distance.pdist(corrMat.to_numpy(dtype='float64'),metric='cosine')
l = scipy.cluster.hierarchy.linkage(d,metric='cosine',method='average',optimal_ordering=True)
c = scipy.cluster.hierarchy.fcluster(l,criterion='maxclust',t=20)

hsv = plt.get_cmap('hsv')
colors = hsv(np.linspace(0, 1.0, len(np.unique(c))+1))
colCL = []
for i in c:
    colCL = colCL + [colors[i]]

sns.set_style("white", rc={"font.family":"Helvetica","axes.grid":False})                                                  
sns.set_context("paper", rc={"font.size":1,"axes.titlesize":1,"axes.labelsize":1,"font.family":"Helvetica","xtick.labelsize":1,"ytick.labelsize":1})
cl = sns.clustermap(corrMat.to_numpy(dtype='float64'),row_linkage=l,col_linkage=l,dendrogram_ratio=(0.05,0.05),
               col_colors=[colCL],row_colors=[colCL],xticklabels=corrMat.index,yticklabels=corrMat.index,cmap='RdBu_r',vmin=-1,vmax=1)
cl.cax.set_visible(False)
plt.savefig('/Users/gouink/Documents/Bladder/Visium/genecorrelation_network_heatmap.svg')
plt.savefig('/Users/gouink/Documents/Bladder/Visium/genecorrelation_network_heatmap.png',dpi=600)

unique, counts = np.unique(c, return_counts=True)
counts = dict(zip(unique, counts))
counts = sorted(counts.items(), key=lambda x: x[1], reverse=True)
counts

for i in counts:
    GOI = corrMat.index[c==i[0]].tolist()
    if len(GOI)>0:
        sc.tl.score_genes(adata,gene_list=GOI,use_raw=False,score_name='clust'+str(i[0]))
        
for i,j in enumerate(counts):
    GOI = corrMat.index[c==j[0]].tolist()
    tmp = pd.DataFrame(GOI,columns=['genename'])
    tmp['cluster'] = 'clust'+str(j[0])
    if i == 0:
        df = tmp
    else:
        df = df.append(tmp)
df.to_csv('/Users/gouink/Documents/Bladder/Visium/genecorrelation_network_genes.csv')

In [ ]:
##score modules and plot module score on section

mods = pd.read_csv('/Users/gouink/Documents/Bladder/Visium/genecorrelation_network_genes.csv')
clust = mods.cluster.unique().tolist()
for c in clust:
    GOI = mods[(mods.cluster==c)].genename.tolist()
    sc.tl.score_genes(adata,gene_list=GOI,use_raw=False,score_name=c)
    
    
GOI = ['clust2','clust11','clust18','clust9','clust16','clust17']
fig,axs = plt.subplots(nrows=len(GOI),ncols=len(samples),sharex=False,sharey=False,figsize=(8,8))

limits = pd.DataFrame(index=GOI,columns=['min','max'])
for g in GOI:
    limits.loc[g,'min'] = adata.obs[g].min()
    limits.loc[g,'max'] = adata.obs[g].max() 

for i,s in enumerate(samples):
    for j,g in enumerate(GOI):
        tmp = adata[(adata.obs.sampleID==s),:].copy()
        a = sc.pl.spatial(tmp,color=g,library_id=s,color_map='RdYlBu_r',legend_loc=None,
                          img_key='hires',size=1.5,vmin=limits.loc[g,'min'],vmax=limits.loc[g,'max'],
                          title=g,show=False,ax=axs[j,i])
        
plt.savefig('/Users/gouink/Documents/Bladder/Visium/correlation_network_scores_v2.svg')

In [ ]:
###specific gene expression around spots

##score modules
mods = pd.read_csv('/Users/gouink/Documents/Bladder/Visium/genecorrelation_network_genes.csv')
clust = mods.cluster.unique().tolist()
for c in clust:
    GOI = mods[(mods.cluster==c)].genename.tolist()
    sc.tl.score_genes(adata,gene_list=GOI,use_raw=False,score_name=c)

###make circle plots
GOI = ['clust2','clust11','clust18','clust9','clust16','clust17']
GOI2 = ['CD3E','CD3D','CD247','CD3G','CD8A','CD8B','CD4','PDCD1','LAG3','HAVCR2','TIGIT','GZMB','ITGA1','CXCL13','ITGAE',
       'CD274','PDCD1LG2','ITGAM','ITGAX','MRC1','CD80','CD86','MZB1','IGHG1','IGHA1','IGHM']
rings = 4
tmp = adata

fig, ax = plt.subplots()

for z,h in enumerate(GOI2):
    
    means = np.zeros((len(GOI),rings+1))
    
    for i,g in enumerate(GOI):

        t = np.quantile(tmp.obs[g].to_numpy(dtype='float64'),q=(0.05,0.95))
        ROI = tmp[(tmp.obs[g]>t[1])].obs.index

        df = np.zeros((len(ROI),rings+1))

        for j,r in enumerate(ROI):

            x = tmp[r].obs.array_row.item()
            y = tmp[r].obs.array_col.item()

            df[j,0] = tmp[r,h].X.toarray().item()

            for k in np.arange(rings):

                xlist = [x-(k+1),x-(k+1),x,x,x+(k+1),x+(k+1)]
                ylist = [y+(k+1),y-(k+1),y+(k+2),y-(k+2),y+(k+1),y-(k+1)]
                coords = zip(xlist,ylist)
                tmp2 = 0
                counter = 0
                for x,y in coords:
                    nn = tmp[((tmp.obs.array_row==x)&(tmp.obs.array_col==y))]
                    if len(nn)>0:
                        tmp2 += nn[:,h].X.toarray().item()
                        counter += 1

                if counter > 0:
                    df[j,(k+1)] = tmp2/counter

        means[i,:] = np.mean(df[:,:],axis=0)
        print(g)

    minima = np.min(means)
    maxima = np.max(means)

    norm = matplotlib.colors.Normalize(vmin=minima, vmax=maxima, clip=True)
    mapper = cm.ScalarMappable(norm=norm, cmap='viridis')

    for i in np.arange(means.shape[0]):
        for j in np.arange(means.shape[1]):
            clr = mapper.to_rgba(means[i,j])
            if (j==0): 
                circle1 = plt.Circle((i*2, z*2), radius=0.1+(0.1*j), color=clr,fill=True)
            else:
                circle1 = plt.Circle((i*2, z*2), radius=0.1+(0.1*j), color=clr,fill=False)
            _= ax.add_artist(circle1)
            
    print(h)
plt.xlim(-1,2*len(GOI))
plt.ylim(-1,2*len(GOI2))
plt.yticks(ticks=np.arange(start=0,stop=2*len(GOI2),step=2),labels=GOI2,fontsize=4)
plt.xticks(ticks=np.arange(start=0,stop=2*len(GOI),step=2),labels=GOI,fontsize=4)
ax.set_aspect('equal')
plt.grid(b=None)
plt.savefig('/Users/gouink/Documents/Bladder/Visium/genecluster_expr_radius_combined.svg')
plt.close()